# Import

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from torch.optim import AdamW

batch_size = 4

# eval summarize, -1-, summary

## Create test Dataset and Dataloader

In [2]:
tokenizer = T5Tokenizer.from_pretrained("kiri-ai/t5-base-qa-summary-emotion")

class JsonlDataset(Dataset):
    def __init__(self, filename, is_test=False):
        self.data = [json.loads(line) for line in open(filename, 'r', encoding='utf-8')]
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def getMerged(self, question, options, article):
        question = question.replace('@placeholder', '_1_')
        options_str = ', '.join(f'{options[i]}' for i in range(5))     
        return f'summarize: Summary<{question}>\nGiven Options<{options_str}>\nAnd Article<{article}>'
    
    def getLabel(self, question, options, label):
        answer = f"{options[label]}"
        return question.replace('@placeholder', answer)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        article = item['article']
        question = item['question']
        options = [item[f'option_{i}'] for i in range(5)]
        
        merged = self.getMerged(question, options, article)
        
        in_tokenize = tokenizer(
            merged,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        input_ids = in_tokenize['input_ids'].flatten()
        attention_mask = in_tokenize["attention_mask"].flatten()
        
        if self.is_test:
            return {
                'article': article,
                'question': question,
                'options': options,
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'merged': merged
            }
        else:
            out_tokenize = tokenizer(
                self.getLabel(question, options, item['label']),
                add_special_tokens=True,
                max_length=150,
                return_tensors="pt",
                padding='max_length'
            )
            
            label = out_tokenize['input_ids'].flatten()
            
            return {
                'article': article,
                'question': question,
                'options': options,
                'label': label,
                'input_ids': input_ids,
                'attention_mask': attention_mask, 
                'label': label,
                'merged': merged
            }

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
task1_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
def eval(model, test_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)
    model.eval()
    
    total = 0
    correct = 0
    for i, batch in enumerate(test_loader):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)
            
            tokens = model.generate(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                max_length=150
            )

            gt = [
                tokenizer.decode(
                    label,
                    skip_special_tokens=True,
                ) for label in label
            ]

            preds = [tokenizer.decode(token, skip_special_tokens=True) for token in tokens]
            
            if i == 0:
                for i, token in enumerate(tokens):
                    print(gt[i])
                    print(preds[i])
                    print('\n')

            total += len(gt)
            correct += sum(gt[i] == preds[i] for i in range(len(preds)))

    print(correct / total)

## Evaluation Task1, Cross1, Task2, Cross2

In [5]:
config = T5Config.from_pretrained("kiri-ai/t5-base-qa-summary-emotion")

model = T5ForConditionalGeneration.from_pretrained('../input/semevalmodelt5/task1_1_summary_best.bin', config=config)
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('../input/semevalmodelt5/task2_1_summary_best.bin', config=config)
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
A US - based media advocacy group has condemned the sentencing by a court in Iran of four reformist journalists to between five and 10 years in prison.
A US - based media advocacy group has condemned the sentencing by a court in Iran of four reformist journalists to between five and 10 years in prison.


A proposed " garden bridge " over the River Thames has been labelled as a " vanity project " of the Mayor of London, a Labour party peer has said.
A controversial " garden bridge " over the River Thames has been labelled as a " vanity project " of the Mayor of London, a Labour party peer has said.


Spain's civil guard force has caught a driver travelling in the high - occupancy lane of a busy road near Madrid, accompanied only by a child - sized doll.
Spain's civil guard force has caught a driver travelling in the high - occupancy lane of a busy road near Madrid, accompanied only by a child - sized doll.


Dozens of families have been told they " may never be able to enter the

## Evaluation without fine tuning

In [6]:
model = model.from_pretrained('kiri-ai/t5-base-qa-summary-emotion')
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('kiri-ai/t5-base-qa-summary-emotion')
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
Gemma Arterton says the character in her latest film is a feminist who " doesn't know it yet ".
Their Finest tells the story of filmmakers making a film to boost morale during World War Two. Film-maker Scherfig's fifth film set in the UK. Arterton plays Catrin Cole, a screenwriter hired to bring a female perspective to war films.


It is a challenge which confronts every first minister. What, they ponder, will my opponents ask when I face them at weekly questions?
Nicola Sturgeon arrived today expecting a raft of constituency issues. She was not disappointed as MSPs quizzed her on council reform. She was not disappointed as she was not prepared for a wider range of topics.


BP has agreed to sell its interest in a major North Sea gas pipeline system to a private equity firm for £ 324 m.
BP sells its 36.22% share in the Central Area Transmission System to a private equity firm. The system includes a 250-mile long pipeline between the Everest field and a gas terminal at Teesside 

# cloze, -3-, i-option[i]

In [7]:
class JsonlDataset2(JsonlDataset):
    def __init__(self, filename, is_test=False):
        super().__init__(filename, is_test)

    def getMerged(self, question, options, article):
        options_str = ', '.join(f'{i}-{options[i]}' for i in range(5))
        question = question.replace('@placeholder', '_3_')
        return f'cloze: Summary<{question}>\nGiven Options<{options_str}>\nAnd Article<{article}>'
        
    def getLabel(self, question, options, label):
        label = f"{label}-{options[label]}"
        return label

In [8]:
task1_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = model.from_pretrained('../input/semevalmodelt5/task1_cloze_option_best.bin', config=config)
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('../input/semevalmodelt5/task2_cloze_option_best.bin', config=config)
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
0-glory
0-glory


0-reportedly
4-successfully


1-widespread
3-corroborate


1-latest
0-incredible


0.247
Task1 Cross: 
1-body
1-body


1-voice
1-voice


0-inspiration
0-inspiration


3-continent
1-island


0.197
Task2: 
3-Formula
2-world


3-regeneration
4-restoration


4-community
4-community


1-deal
0-deals


0.149
Task2 Cross: 
0-belong
3-afford


2-major
2-major


2-crisis
2-crisis


3-verdicts
4-languishing


0.221
